In [41]:
import numpy as np
import pandas as pd
import tqdm

In [45]:
def create_filelist(trial_files, save_path, text_label_file, metainfo_path):
    with open(text_label_file, "r") as f:
        texts = [t.rstrip() for t in f.readlines()]

    df = pd.read_csv(metainfo_path)

    ecog_fileprefix="hgr_"
    idx2text={}
    idx2fileidx={}
    idx2filename={}
    idx2textidx={}
    for fileidx, (block_idx, trial_idx, text, label) in enumerate(zip(df['block_num'].values,
                                              df['trial_num'].values,
                                              df['gt_text'].values,
                                                                     df['gt_idx'])):
        idx = f'B{block_idx}_{trial_idx:05d}'
        idx2text[idx]=text
        idx2fileidx[idx]=fileidx
        idx2filename[idx] = f'{ecog_fileprefix}{block_idx}_{trial_idx}_{label}.npy'
        idx2textidx[idx] = texts.index(text)

    with open(trial_files, "r") as f:
        trials = ["_".join(t.rstrip().split("_")[1:3]) for t in f.readlines()]
    
    def idx2hbunits(idx):
        hb_dir = "/data/cheoljun/b3_audio_scale-2/unit_label/hubert-l6_km100/tm1k"

        hb = np.load((hb_dir+f"/{idx2textidx[idx]}.npy"))

        return " ".join(hb.astype(str))

    lines = []
    for idx in tqdm.tqdm(trials):
        line = "|".join([idx2filename[idx],idx2text[idx], idx2hbunits(idx)])
        lines.append(line)

    with open(save_path, "w") as f:
        for line in lines:
            f.write(line+"\n")
    return lines

In [43]:
text_label_file = '/data/cheoljun/b3_misc/audio_info/tm1k_text_20240107.txt'
train_files = '/home/cheoljun/rnnt.streaming.anyunit/outputs/2024-01-25/18-53-33/train.txt'
val_files = '/home/cheoljun/rnnt.streaming.anyunit/outputs/2024-01-25/18-53-33/test.txt'
test_files = "/data/common/b3_paper/packaged_data/streaming_paper_data/tm1k_mimed_slow/test_demoday_ecog_tm1k_synthesis.txt"
metainfo_path = "/data/cheoljun/b3_misc/tm1k_streamer_paper_final_session_dataframe.csv"

In [4]:
save_path = "/data/common/bravo_streaming_data/filelist/tm1k_train.txt"

In [46]:
_=create_filelist(train_files, "/data/common/bravo_streaming_data/filelist/tm1k_train.txt", text_label_file, metainfo_path)

100%|██████████████████████████████████████████████████████████████| 23378/23378 [01:20<00:00, 289.48it/s]


In [48]:

_=create_filelist(val_files, "/data/common/bravo_streaming_data/filelist/tm1k_dev.txt", text_label_file, metainfo_path)
_=create_filelist(test_files, "/data/common/bravo_streaming_data/filelist/tm1k_test_withfeedback.txt", text_label_file, metainfo_path)

100%|██████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 306.12it/s]


In [49]:
_=create_filelist("/data/cheoljun/b3_misc/test_demoday_ecog_tm1k_synthesis_first50.txt",
                  "/data/common/bravo_streaming_data/filelist/tm1k_test50_withfeedback.txt", text_label_file, metainfo_path)

100%|███████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 4239.33it/s]


In [50]:
_=create_filelist("/data/common/b3_paper/packaged_data/streaming_paper_data/tm1k_mimed_slow/test_demoday_nofeedback_tm1k_synthesis.txt",
                  "/data/common/bravo_streaming_data/filelist/tm1k_test50_withoutfeedback.txt", text_label_file, metainfo_path)

100%|███████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 4285.85it/s]
